In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import minmax_scale

from scipy.stats import norm, gaussian_kde

# Download the date
https://archive.ics.uci.edu/ml/machine-learning-databases/00577/codon_usage.csv.zip


In [ ]:
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00577/codon_usage.csv.zip

In [ ]:
# !unzip codon_usage.csv.zip

# Data Preprocessing

In [ ]:
def read_data(file_name):
    """
    Reads in a csv file and returns a dataframe
    """
    return pd.read_csv(file_name, low_memory=False)

dataset = read_data('codon_usage.csv')
df = dataset.copy()

In [ ]:
# dataset = read_data('codon_usage.csv')
dataset.describe()

In [ ]:
dataset

In [ ]:
# dataset.dropna(inplace=True)

dataset['Kingdom'].value_counts()

In [ ]:
print(dataset.isnull().sum())

In [ ]:
dataset['Kingdom'].value_counts().plot(kind='bar')

In [ ]:
classes_dict = {    'arc': 0, 'bct': 1, 'phg': 2, 'plm': 3, 
                    'pln': 4, 'inv': 5, 'vrt': 6, 'mam': 7,
                    'rod': 8, 'pri': 9, 'vrl':10
                }

In [ ]:
dataset['Kingdom'] = dataset['Kingdom'].map(classes_dict)

In [ ]:
# dataset['Kingdom'].value_counts().plot(kind='bar')

In [ ]:
dataset = dataset.drop(dataset[dataset['SpeciesID']==353569].index)
dataset = dataset.drop(dataset[dataset['SpeciesID']==1238].index)


In [ ]:
dataset['UUU'] = dataset['UUU'].astype(float)
dataset['UUC'] = dataset['UUC'].astype(float)
# dataset = minmax_scale(dataset)
cols = dataset.select_dtypes(np.number).columns

num_columns_list = list(cols)
# num_columns_list

num_columns_list.remove('Kingdom')
num_columns_list.remove('DNAtype')
num_columns_list.remove('SpeciesID')
num_columns_list.remove('Ncodons')
# num_columns_list

In [ ]:
# for num_column in num_columns_list:
#     dataset[num_column] = minmax_scale(dataset[num_column])

dataset.head()

In [ ]:
X = dataset.iloc[:, 5:].values.astype(float)
y = dataset.iloc[:, 0]

In [ ]:
X

In [ ]:
class DataGenerator:

    def __init__(self, X, y, sampleSize=16):
        self.X = X
        self.y = y
        self.sampleSize = sampleSize
        if self.sampleSize <= 0 and self.sampleSize > len(self.X):
            return TypeError("sampleSize must be between 0 and ", len(X))



    def split_data(self, test_size=0.3, random_state=1):
        self.X_train, self.X_rem, self.y_train, self.y_rem = tts(self.X, self.y, test_size=test_size, random_state=random_state)
        self.X_test, self.X_val, self.y_test, self.y_val = tts(self.X_rem, self.y_rem, test_size=0.5, random_state=random_state)
        return self.X_train, self.X_test, self.y_train, self.y_test
    

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
data_generator = DataGenerator(y=y, X=X)
X_training, X_testing, y_training, y_testing = data_generator.split_data()

In [ ]:
class MultiClassClassification:
  def __init__(self, X_train, X_test, y_train, y_test, laplace_smoothing_value, laplace, total_classes=2):
    self.laplace_smoothing_value = laplace_smoothing_value
    self.laplace = laplace
    self.total_classes = total_classes
    self.X_train = X_train
    self.X_test = X_test
    self.y_train = y_train
    self.y_test = y_test
    self.graylist =[]
    self.laplaceProbabilitylist = []
    self.fitFeatures= []


  def fitDistribution(self, data):

    # mean = np.mean(data) # mean of the data
    # std = np.std(data) # standard deviation of the data
    # dist = norm(mean, std) # create a normal distribution with the mean and standard deviation 

    return gaussian_kde(data) # return the distribution

  def evaluate(self, y, y_predicted, desired_class):
    '''

      Takes original classes and predicted classes as input

      Return the values of precision, recall and accuracy
    
    '''
    y = ( y==desired_class )
    y_predicted = ( y_predicted == desired_class)

    precision = (y&y_predicted).sum() / y_predicted.sum()
    recall = (y&y_predicted).sum() / y.sum()
    accuracy = (y==y_predicted).sum() / y.size


    return precision, recall, accuracy


  def laplaceSmoothing(self, classes, length):
    smooths = np.empty(self.total_classes, dtype=object) # create an empty list to store the smoothed values
    for i in range(length): 
      # add the laplace smoothing value to the class and divide by the size of the class plus the laplace smoothing value times the number of features
      smooths[i] = (classes[i] + self.laplace_smoothing_value) / ( (classes[i].size) + self.laplace_smoothing_value * self.n_features) 
    
    return smooths 



  # def probability(self, X, dist1, dist2, prior):
  #   return prior * ( dist1.pdf(X[0]) *  dist2.pdf(X[1]))


  def probability(self, data):

    py = np.ones(self.total_classes) # create an array of ones to store the probabilities


    for i in range(self.n_features):
      for j in range(self.total_classes):
        # if 'X'+str(i)+str(j) in self.graylist:
        #   print('already in graylist')
        #   print('X'+str(i)+ '' + str(j))
        # else:
        #   self.graylist.append('X'+str(i)+str(j))
          
        py[j] *= self.features['X'+str(i)+str(j)].pdf(data[i])
        

    for i in range(self.total_classes):
      py[i] *= self.prior[i]
    
    return  py



  def laplaceProbability(self, data):
    
    py = np.ones(11) # create an array of ones to store the probabilities
    pdf = np.ones(self.total_classes) # create an array of ones to store the PDF values

    for i in range(self.n_features):
      for j in range(self.total_classes):
        # print('laplaceProbability','i ', i, ' j ', j)
        # if 'X'+str(i)+str(j) in self.laplaceProbabilitylist:
        #   print('already in laplaceProbabilitylist')
        # else:
        #   self.laplaceProbabilitylist.append('X'+str(i)+str(j))
        pdf[j] *= self.features['X'+str(i)+str(j)].pdf(data[i])

      
      smooths = self.laplaceSmoothing(pdf, self.total_classes)
      for j in range(self.total_classes):
        py[j] *= smooths[j]
      
    for i in range(self.total_classes):
      py[i] *= self.prior[i]


    return py




  def fit(self):

    # self.X_train, self.X_test, self.y_train, self.y_test = self.splitData()
    X_training_class = np.empty(self.total_classes, dtype=object)
    for i in range(self.total_classes):
      X_training_class[i] = self.X_train[self.y_train == i]

    self.prior = np.zeros(self.total_classes)
    for i in range(self.total_classes):
      print('setting prior', i)
      self.prior[i] = len(X_training_class[i]) / len(self.X_train)

    


    self.n_features = self.X_train.shape[1]
    # print('Number of features: ', self.n_features)
    self.features = {}
    

    for i in range(self.n_features):
      for j in range(self.total_classes):
        # print('fit features','i ', i, ' j ', j)
        # if 'X'+str(i)+str(j) in self.fitFeatures:
        #   print('already in features')
        # else:
        #   self.fitFeatures.append('X'+str(i)+str(j))
          
        self.features['X'+str(i)+str(j)] = self.fitDistribution(X_training_class[j][:,i])
  


  
  
  def predict(self):
    count = 0
    y_predicted = []
    y = []
    for sample, target in zip(self.X_test, self.y_test):

      if self.laplace:
        py = self.laplaceProbability(sample)
      
      else:
        py = self.probability(sample)

      # print("Model predicted class {} and truth was {}".format(np.argmax(py), target))
      y_predicted.append(np.argmax(py))
      y.append(target)
      # np.append(y_predicted, np.argmax([py0,py1]))
      # np.append(y, target)
      # y = target
      if np.argmax(py) != target :
        count+=1

    # precision, recall, accuracy = self.evaluate(y, y_predicted, desired_class=10)
    # print("precision:", precision)
    # print("recall:", recall)
    # print("accuracy:", accuracy)

    y_predicted = np.array(y_predicted)
    y = np.array(y)
    return y_predicted, y

In [ ]:
clf = MultiClassClassification(X_train=X_training, y_train=y_training, X_test=X_testing, y_test=y_testing, laplace=False, total_classes=11, laplace_smoothing_value=0.5)

In [ ]:
clf.fit()

In [ ]:
y, y_predicted = clf.predict()

In [ ]:
import matplotlib.pyplot as plt

def draw_matrix(true_positive, false_positive, false_negative, true_negative):
  '''
    Draw a confusion matrix.
  '''
  matrix = np.array([[true_positive, false_negative], [false_positive, true_negative]])
  plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.inferno_r)
  plt.colorbar()
  plt.show()


def confusion_matrix(y, y_predicted, desired_class):
  '''
    Takes original classes and predicted classes as input
  '''
  actual = ( y == desired_class )

  predicted = ( y_predicted == desired_class )
  true_positive = (actual & predicted).sum()
  false_positive = (actual & ~predicted).sum()
  false_negative = (~actual & predicted).sum()
  true_negative = (~actual & ~predicted).sum()

  return true_positive, false_positive, false_negative, true_negative

In [ ]:
for class_name in classes_dict.keys():
  print(class_name)
  print(dataset[dataset['Kingdom'] == int(classes_dict.get(class_name))]['Kingdom'].count())
  true_positive, false_positive, false_negative, true_negative = confusion_matrix(y, y_predicted, int(classes_dict.get(class_name)))
  # draw_matrix(true_positive= true_positive, true_negative= true_negative, false_positive= false_positive, false_negative= false_negative)
  print("true positive:", true_positive)
  print("false negative:", false_negative)
  print("false positive:", false_positive)
  print("true negative:", true_negative)
  print("Precision:", true_positive / (true_positive + false_positive))
  print("True Positive rate or Recall:", true_positive / (true_positive + false_negative))
  print("specificity, selectivity or True Negative Rate:", true_negative / (true_negative + false_positive))
  print("Accuracy:", (true_positive + true_negative) / (true_positive + false_positive + false_negative + true_negative))
  print("Balanced accuracy:", (true_positive + true_negative) / (true_positive + false_positive + false_negative + true_negative))
  # print("F1 score:", 2 * ( (true_positive / (true_positive + false_positive)) * (true_positive / (true_positive + false_negative))) / ((true_positive / (true_positive + false_positive)) + (true_positive / (true_positive + false_negative))) )
  # print("MCC:", ( (true_positive * true_negative) - (false_positive * false_negative) ) / ( (true_positive + false_positive) * (true_positive + false_negative) * (true_negative + false_positive) * (true_negative + false_negative) ) )
  print("\n")

In [ ]:
def original_accuracy(y, y_predicted):
  '''
    Takes original classes and predicted classes as input
  '''
  actual = ( y == y_predicted )
  return actual.sum() / len(y)

print(original_accuracy(y, y_predicted))